In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from pymatgen.core.structure import Structure, Lattice
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import CutOffDictNN
from torch_geometric.data import Data
import numpy as np
from omegaconf import OmegaConf
from mofgraph2vec.featurize.mof2doc import MOF2doc
import pandas as pd
from hydra.utils import instantiate
from gensim.models.doc2vec import Doc2Vec

In [3]:
df_vector = pd.read_csv("../data/embedding_example/embedding-h50rl5oi.csv").set_index("type")

In [4]:
config = OmegaConf.load("../conf/config.yaml")
config.mof2vec_data = OmegaConf.load("../conf/mof2vec_data/data.yaml")
config.mof2vec_model = OmegaConf.load("../conf/mof2vec_model/gensim.yaml")

In [5]:
config.mof2vec_data.data.cif_path = ["../data/cifs/rsm/"]

In [6]:
config.mof2vec_data.data.subsample = 0.005
config.mof2vec_data.data.wl_step = 4

In [7]:
doc = instantiate(config.mof2vec_data.data, seed=config.seed)

In [20]:
documents = doc.get_documents()

100%|██████████| 16/16 [00:00<00:00, 285.70it/s]


In [ ]:
names = [d.tags[0] for d in documents]
word_length = [len(d.words) for d in documents]

In [ ]:
word_length_ordered = [l for _, l in sorted(zip(names, word_length))]

In [ ]:
vector_length = np.sum(df_vector**2, axis=1)

In [9]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(word_length_ordered, vector_length)
plt.xlabel("MOF document length")
plt.ylabel("Embedded vector length")

In [ ]:
np.corrcoef(word_length_ordered, vector_length)

In [16]:
from mofgraph2vec.utils.saving import save_embedding
from hydra.utils import instantiate
from gensim.models.doc2vec import Doc2Vec

In [22]:
model = Doc2Vec(**config.mof2vec_model.gensim)
model.build_vocab(documents)

# Load embedding model

In [11]:
em_model = Doc2Vec.load("../experiments/workflow/wandb/offline-run-20230509_154052-ygb7sowe/tmp/embedding_model.pt")

In [12]:
em_model.corpus_count

20375

In [13]:
em_model.docvecs.most_similar(["qmof-8a95c27"])

/var/folders/gt/tsyp2wjd1ng0pkr1tf52kn600000gn/T/ipykernel_24774/2944465799.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  em_model.docvecs.most_similar(["qmof-8a95c27"])


[('qmof-1c7a916', 0.9535780549049377),
 ('qmof-9c78145', 0.9474985003471375),
 ('qmof-3e7bd15', 0.8505899310112),
 ('qmof-22e7468', 0.7998324036598206),
 ('qmof-12380d1', 0.7977709174156189),
 ('qmof-e2f1ea2', 0.7808641195297241),
 ('qmof-c5d75d5', 0.7623779773712158),
 ('qmof-22ca4f8', 0.7604265809059143),
 ('qmof-f2fac68', 0.759960412979126),
 ('qmof-4a7bb80', 0.7523057460784912)]